<a href="https://colab.research.google.com/github/ramari96/Algoritmos_de_optimizacion_2026/blob/ramiro-algoritmo/Seminario/Seminario_Algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: Emma Otiavén Carracedo - Ramiro Rivas Fernández  <br>
Url: https://github.com/.../03MAIR---Algoritmos-de-Optimizacion---2019/tree/master/SEMINARIO<br>
Problema:
> 1. Sesiones de doblaje <br>
>2. Organizar los horarios de partidos de La Liga<br>
>3. Combinar cifras y operaciones

Descripción del problema: Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible.

(*) La respuesta es obligatoria





                                        

#1.a ¿Cuántas posibilidades hay sin tener en cuenta las restricciones?

### Posibilidades sin tener en cuenta las restricciones

En el problema se nos pide planificar las sesiones por día de manera que minimicemos el gasto de los actores. Si tratamos al problema como un n-upla de 30 elementos (tomas) obtendríamos:

$$T = (T_1,...,T_{30})$$

¿Cuántas formas de ordenar esta n-upla existen?

Sin tener en cuenta las restricciones del problema, las posibilidades se pueden calcular como permutaciones en las que **no se pueden dar repeticiones** y en las que el orden **importa**.

Siguiendo la fórmula de permutación sin repetición y donde el orden importa: $$P_n = n!$$

Para este problema en concreto: $P_{30} = 30!$

#1.b ¿Cuántas posibilidades teniendo en cuenta las restricciones?
### Posibilidades teniendo en cuenta las restricciones

La restricción que se nos plantea es que como máximo se pueden grabar 6 tomas por día, entonces la pregunta pasa a ser: ¿cuántos conjuntos de 6 elementos puedo formar con los elementos de una lista de 30 elementos?

Esto serían combinaciones de 6 elementos entre 30 donde no se puede repetir e importa el orden en el que están dichos elementos:

$$C\binom{n}{k} = \frac{n!}{k!(n-k)!}$$

Para nuestro caso: $C\binom{30}{6} = \frac{30!}{6! 24!} = 593775$




#2. ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, argumentalo)


Se propone modelar a las tomas como un array de treinta elementos en el que cada elemento representa una toma.

Este modelado unidimencional del problema nos ayuda a disminuir la cantidad de operaciones que se deben hacer. Esto se debe a que la matriz de organización del rodaje provista es fija, no debemos organizarla ni estructurarla por lo que no es parte del problema. La usaremos como consulta para los costes de nuestra función objetivo pero no forma parte de la solución.

La estructura de datos sera por tanto un arreglo del tipo:

$$T = [1,2,3,...,30]$$ donde cada $T_i$ representa el número de una de las tomas.


#3. :¿Cuál es la función objetivo? ¿Es un problema de maximización o minimización?

Para nuestro problema, debemos tener en cuenta que elementos aportan datos para la solución, haciendo un análisis de la letra obtenemos:

- 30 tomas que debemos grabar.
- 10 actores que participan en las tomas, cada uno aparecerá en la matriz de organización como 1 si aparece en la toma, o 0 en caso contrario.
- Los actores cobran por día y no por toma, esto quiere decir que si un actor aparece en todas las tomas de un día cobrará igual que si aparece en sólo 1.

Con esto en mente, lo que queremos es calcular cuánto nos costará el rodaje, sabiendo que sólo podemos grabar un máximo de 6 tomas por día.

$$F = \sum_{i=1}^{n} CosteDia_i$$

Es un problema de *minimización* ya que lo que queremos es minimizar el coste por día de rodaje.







#4. Diseña un algoritmo para resolver el problema por fuerza bruta

In [ ]:
import pandas as pd
import numpy as np
import itertools

###################################################################################################################
################################## PARTE 1: TRATADO DE DATOS DEL PROBLEMA #########################################
###################################################################################################################

#Lectura de datos y cargado del dataframe que contiene la matriz de organización del rodaje
data_url = "https://docs.google.com/spreadsheets/d/1Ipn6IrbQP4ax8zOnivdBIw2lN0JISkJG4fXndYd27U0/export?format=csv"
matriz_de_rodaje = pd.read_csv(data_url)

#Transformación de los encabezados de las columnas a Actor 1, ..., Actor10
matriz_de_rodaje = matriz_de_rodaje.rename(columns={
    'Actor': 'Actor 1',
    'Unnamed: 2': 'Actor 2',
    'Unnamed: 3': 'Actor 3',
    'Unnamed: 4': 'Actor 4',
    'Unnamed: 5': 'Actor 5',
    'Unnamed: 6': 'Actor 6',
    'Unnamed: 7': 'Actor 7',
    'Unnamed: 8': 'Actor 8',
    'Unnamed: 9': 'Actor 9',
    'Unnamed: 10': 'Actor 10',
})

#Procesado de las tomas para convertirlas en conjuntos de actores unicos (así podemos saber el coste de cada toma)
tomas = matriz_de_rodaje.columns[1:]
tomas_actores = []

#Limpieza del dataframe de aquellas filas que no aportan lo que necesitamos
matriz_de_rodaje = matriz_de_rodaje.drop(index=0).drop(index=[31,32])

for _, row in matriz_de_rodaje.iterrows():
  actores_unicos = set(actores for actores in tomas if row[actores]==1) #Se busca que row[actores]==1 porque eso quiere decir que el actor participa en la toma
  tomas_actores.append(actores_unicos)


###################################################################################################################
################################## PARTE 2: FUNCIÓN DE COSTE PARA EL PROBLEMA #####################################
###################################################################################################################

def coste_total(tomas):
  #Cálculo del coste
  coste = 0

  #Se recorre la solución actual de 6 en 6 (esto es por la restricción de los días)
  for i in range(0, len(tomas), 6):

    #Vemos que tomas tocan en el día actual (selecciona las tomas de 6 en 6 en el orden que aparecen)
    bloque_actual = tomas[i:i+6]
    print (bloque_actual)

    actores_del_bloque = set()

    for toma in bloque_actual:
      actores_de_la_toma = tomas_actores[toma - 1] #Porque las tomas van de 1-30 y los indexes de 0-29
      actores_del_bloque.update(actores_de_la_toma)

    coste += len(actores_del_bloque) #Esto es porque se sabe que los actores cobran 1 por día
  print(f'coste total de la planificación: {coste}')
  return coste

###################################################################################################################
################################## PARTE 3: RESOLUCIÓN POR FUERZA BRUTA ###########################################
###################################################################################################################

def fuerza_bruta(numero_tomas):
  '''
  Resuelve el problema para un número limitado de tomas, esto es porque de abarcar las 30 tomas tendríamos demasiadas
  combinaciones posibles
  '''

  #Creo la solución inicial T para el problema con la cantidad de tomas indicada
  S = np.random.permutation(np.arange(1,numero_tomas+1))

  #inicializo mejor_coste y mejor_solucion
  mejor_coste = float('inf')
  mejor_solucion = []

  for permutacion in itertools.permutations(S):
    #defino la solucion actual y el coste actual como S y el coste implicado en S
    solucion_actual = S
    coste_actual = coste_total(S)

    #si el coste calculado es mejor que el mejor guardado, lo cambio
    if coste_actual < mejor_coste:
      mejor_coste = coste_actual
      mejor_solucion = solucion_actual

    return mejor_solucion, mejor_coste


###################################################################################################################
################################## PARTE 3: RESOLUCIÓN POR FUERZA BRUTA ###########################################
###################################################################################################################

solucion, coste = fuerza_bruta(8)
print(f'La solución es {solucion} y el coste asociado es {coste}')
print(tomas_actores)




[3 5 7 6 8 2]
[1 4]
coste total de la planificación: 15
La solución es [3 5 7 6 8 2 1 4] y el coste asociado es 15
[{'Actor 4', 'Actor 1', 'Actor 3', 'Actor 5', 'Actor 2'}, {'Actor 4', 'Actor 3', 'Actor 5'}, {'Actor 7', 'Actor 5', 'Actor 2'}, {'Actor 7', 'Actor 1', 'Actor 8', 'Actor 2'}, {'Actor 4', 'Actor 8', 'Actor 2'}, {'Actor 4', 'Actor 1', 'Actor 5', 'Actor 2'}, {'Actor 4', 'Actor 1', 'Actor 5', 'Actor 2'}, {'Actor 1', 'Actor 6', 'Actor 2'}, {'Actor 4', 'Actor 1', 'Actor 2'}, {'Actor 1', 'Actor 6', 'Actor 2', 'Actor 9'}, {'Actor 1', 'Actor 3', 'Actor 5', 'Actor 8', 'Actor 2'}, {'Actor 4', 'Actor 1', 'Actor 3', 'Actor 6', 'Actor 2'}, {'Actor 4', 'Actor 1', 'Actor 5'}, {'Actor 1', 'Actor 3', 'Actor 6'}, {'Actor 7', 'Actor 1', 'Actor 2'}, {'Actor 4', 'Actor 10'}, {'Actor 1', 'Actor 3'}, {'Actor 3', 'Actor 6'}, {'Actor 1', 'Actor 3'}, {'Actor 4', 'Actor 1', 'Actor 3', 'Actor 5'}, {'Actor 8', 'Actor 6'}, {'Actor 4', 'Actor 1', 'Actor 3', 'Actor 2'}, {'Actor 1', 'Actor 3'}, {'Actor 3', 

#5. Calcula la complejidad del algoritmo por fuerza bruta

Para el cálculo de la complejidad del algoritmo *fuerza_bruta* debemos considerar:
1. *for de itertools:* calcula todas las permutaciones posibles sin repetición para un conjunto de n elementos, por lo que sabemos que su complejidad algorítmica es $O(n!)$
2. *coste de valores*: al calcular el coste de la lista de longitud n tenemos un coste operacional lineal de $O(n)$

Por lo que nuestro algoritmo tiene una complejidad total de $O(n*n!)$. Esto hace que no sea escalable ya que para valores muy grandes de n el factorial crece de forma exponencial haciendo que el computo de todas las posibles iteraciones aumente muy rápido.

(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta

(*)Calcula la complejidad del algoritmo

Respuesta

Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Respuesta

Aplica el algoritmo al juego de datos generado

Respuesta

Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Respuesta

Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta